In [1]:
import torch
import torch.nn as nn
from collections import deque
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from models.VGG import VGG
from pgd import PGD
import matplotlib.pyplot as plt

# if importing locals
from rails import RAILS
from aise import AISE
import aise


CIFAR_CONFIGS = {
    "start_layer": 1,
    "n_class": 10,
    "aise_params": [
        {"hidden_layer": 2, "sampling_temperature": 1, "max_generation": 10, "mut_range": (.005, .015)},
        {"hidden_layer": 3, "sampling_temperature": 10, "max_generation": 5, "mut_range": (.005, .015)}
    ]
}

ROOT = "./datasets"
TRANSFORM = transforms.ToTensor()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = datasets.CIFAR10(root=ROOT, download=True, train=True, transform=TRANSFORM)
test_data = datasets.CIFAR10(root=ROOT, download=True, train=False, transform=TRANSFORM)
#     train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
#     test_loader = DataLoader(test_data, batch_size=1024, shuffle=False)

x_train = (torch.FloatTensor(train_data.data[:50000])/255).permute(0,3,1,2)
y_train = torch.LongTensor(train_data.targets[:50000])

model = VGG()
model.load_state_dict(torch.load(
    "./model_weights/cifar_vgg16.pt", map_location=DEVICE
)['state_dict'])
model.eval()

pgd = PGD(eps=8/255,step=2/255,max_iter=10, batch_size=256)

x_batch = (torch.FloatTensor(test_data.data[:1000])/255).permute(0,3,1,2)
y_batch = torch.LongTensor(test_data.targets[:1000])

x_adv = pgd.generate(model, x_batch, y_batch, device=DEVICE)

pred_clean = model(x_batch.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
clean_acc = (pred_clean == y_batch).float().mean().item()
print("Clean accuracy is {}.".format(clean_acc))

pred_adv = model(x_adv.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
adv_acc = (pred_adv == y_batch).float().mean().item()
print("Adversarial accuracy (CNN) is {}.".format(adv_acc))

Files already downloaded and verified
Files already downloaded and verified
Clean accuracy is 0.8759999871253967.
Adversarial accuracy (CNN) is 0.32199999690055847.


In [2]:
# %%timeit
# """
# rails constructor execution time
# """

# rails = RAILS(model, CIFAR_CONFIGS, x_train, y_train)

In [3]:
%load_ext line_profiler
"""
load the line profiler
"""


%load_ext memory_profiler
"""
load the memory profiler 
"""

'\nload the memory profiler \n'

In [4]:
"""
rails constructor line by line profiling
"""
    
%prun 

rails = RAILS(model, CIFAR_CONFIGS, x_train, y_train)

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [5]:
"""
rails predict
"""
    
%lprun -f rails.predict rails.predict(x_adv.to(DEVICE)).argmax(axis=1)

Timer unit: 1e-06 s

Total time: 296.586 s
File: /home/cstansbu/RAILS/rails.py
Function: predict at line 73

Line #      Hits         Time  Per Hit   % Time  Line Contents
    73                                               def predict(self, x):
    74         1         68.0     68.0      0.0          with torch.no_grad():
    75         3      71631.0  23877.0      0.0              x_start = torch.cat([
    76                                                           self._model.to_start(x[i:i + self.batch_size].to(DEVICE)).cpu()
    77         1          8.0      8.0      0.0                  for i in range(0, x.size(0), self.batch_size)
    78         1          3.0      3.0      0.0              ], dim=0)
    79         1         52.0     52.0      0.0          pred = np.zeros((x_start.size(0), self.n_class))
    80         3          9.0      3.0      0.0          for aise in self.aises:
    81         2  296514495.0 148257247.5    100.0              pred = pred + aise(x_start)
 

In [7]:
"""
aise profile
"""

x_start = torch.cat([
    rails._model.to_start(x_adv[i:i + rails.batch_size].to(DEVICE)).cpu()
    for i in range(0, x_adv.size(0), rails.batch_size)
], dim=0)

params = {
    "hidden_layer": 2, 
    "sampling_temperature": 1, 
    "max_generation": 10, 
    "mut_range": (.005, .015)
}
    
aise = AISE(model=rails._model, 
     x_orig=rails.x_train, 
     y_orig=rails.y_train, 
     dataset = "cifar", 
     **params)


%lprun -f  aise.clonal_expansion aise.clonal_expansion(x_start)

Timer unit: 1e-06 s

Total time: 318.572 s
File: /home/cstansbu/RAILS/aise.py
Function: clonal_expansion at line 360

Line #      Hits         Time  Per Hit   % Time  Line Contents
   360                                               def clonal_expansion(self, ant, y_ant=None):
   361         1         23.0     23.0      0.0          logger.info("Clonal expansion starts...")
   362         1      10967.0  10967.0      0.0          ant_tran = self._hidden_repr_mapping(ant.detach())
   363         1   69096436.0 69096436.0     21.7          nbc_ind = self._query_nns_ind(ant_tran.detach().cpu().numpy())
   364         2  249464112.0 124732056.0     78.3          mem_bcs, mem_labs, pla_bcs, pla_labs, ant_logs = self.generate_b_cells(
   365         1        149.0    149.0      0.0              ant.flatten(start_dim=1),
   366         1          1.0      1.0      0.0              ant_tran,
   367         1          0.0      0.0      0.0              nbc_ind,
   368         1          0.0   

In [9]:
"""
profile memory b-cell generation
"""
ant = x_start
ant_tran = aise._hidden_repr_mapping(ant.detach())
nbc_ind = aise._query_nns_ind(ant_tran.detach().cpu().numpy())


%lprun -f aise.generate_b_cells aise.generate_b_cells(ant.flatten(start_dim=1), ant_tran, nbc_ind, None)



Timer unit: 1e-06 s

Total time: 237.348 s
File: /home/cstansbu/RAILS/aise.py
Function: generate_b_cells at line 237

Line #      Hits         Time  Per Hit   % Time  Line Contents
   237                                               def generate_b_cells(self, ant, ant_tran, nbc_ind, y_ant=None):
   238         1          9.0      9.0      0.0          assert ant_tran.ndim == 2, "ant: 2d tensor (n_antigens,n_features)"
   239                                           
   240                                                   # make sure data and indices are in the same class
   241         1         13.0     13.0      0.0          if isinstance(self.x_orig, torch.Tensor):
   242         1        116.0    116.0      0.0              nbc_ind = torch.LongTensor(nbc_ind)
   243                                           
   244         1          2.0      2.0      0.0          pla_bcs = []
   245         1          2.0      2.0      0.0          pla_labs = []
   246         1          4.0   